# resources

BSI syntax:
- https://engineering.paypalcorp.com/confluence/display/ARSBIGDATA/BSI+-+Big+data+Scoring+Infrastructure

# run BSI in notebook

- follow this steps: https://engineering.paypalcorp.com/confluence/display/BIGDATA/BSI+on+Notebook+GCP
- note:
    - run with pyspark kernel
    - use paypal notebook, not gpu notebook or cosmos
    - when shutting down notebook kernel, created GCP cluster will also be deleted



# config and run BSI code

## create a cluster

In [1]:
%%configuration --project ccg24-hrzana-gds-focus --dataproc -fa https://artifactory.paypalcorp.com/artifactory/big-data-platform/misc/bsi/2.6.0-SNAPSHOT/notebook_create_cluster_template.json --args cluster_name=guxia-bsi-2,project_id=ccg24-hrzana-gds-focus,cluster_size=small
{
    "computeCommonConfig": {
        "metadata": {
            "PIP_PACKAGES": "pandas==0.23.0 scipy==1.1.0", 
            "CONDA_PACKAGES": "tensorflow"
        }
    }
}

Read configuration from path https://artifactory.paypalinc.com/artifactory/big-data-platform/misc/bsi/2.6.0-SNAPSHOT/notebook_create_cluster_template.json
Creating cluster with configuration:
{
    "projectId": "ccg24-hrzana-gds-focus",
    "secondaryWorkers": {
        "DiskType": "pd-standard",
        "sizeInGb": 500,
        "type": "n1-highmem-8"
    },
    "autoScalePolicyUri": "projects/ccg24-hrzana-gds-focus/regions/us-west4/autoscalingPolicies/bsi_policy_small",
    "name": "guxia-bsi-2",
    "image": "1.4",
    "computeCommonConfig": {
        "metadata": {
            "CONDA_PACKAGES": "tensorflow",
            "PIP_PACKAGES": "pandas==0.23.0 scipy==1.1.0"
        }
    },
    "labels": {
        "pp-edp-custom-billing-tag": "bsi",
        "pp-edp-platform": "bsi"
    },
    "workers": {
        "disk": {
            "DiskType": "pd-standard",
            "sizeInGb": 500,
            "numLocalSsds": 4
        },
        "type": "n1-highmem-8"
    },
    "master": {
        "

## connect to cluster

In [3]:
%%url --dataproc --cluster guxia-bsi-2 --project ccg24-hrzana-gds-focus

Now cluster guxia-bsi-2 in project ccg24-hrzana-gds-focus will be used to start session.
Livy endpoint is: https://wrtkfbsf7jf4hfxgqwzfo65die-dot-us-west4.dataproc.googleusercontent.com/livy/v1

## customized config

In [4]:
%configure -fa https://artifactory.paypalcorp.com/artifactory/big-data-platform/misc/bsi/2.6.0-SNAPSHOT/spark_conf_small.json

Read configuration from path https://artifactory.paypalinc.com/artifactory/big-data-platform/misc/bsi/2.6.0-SNAPSHOT/spark_conf_small.json


## get a spark session

In [5]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1689256194764_0001,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.

# run BSI code

In [6]:
%%bsisql
set bsi.gcp.bucket=pypl-bkt-rsh-row-std-gds-focus;

,key,value
0,bsi.gcp.bucket,pypl-bkt-rsh-row-std-gds-focus


In [12]:
# %%bsisql
# -- rename table
# ALTER TABLE ucc_phone_chargeback_production.ucc_phone_chargeback_history RENAME TO ucc_phone_chargeback_production.ucc_phone_chargeback_history_tmp;

In [9]:
%%bsisql
-- query table
select
    running_dt
    , count(*) as num
from ucc_shipping_addr_chargeback_production_verification.ucc_shipping_addr_chargeback_history
group by running_dt
order by running_dt

,running_dt,num
0,20230617,224234924


# show GCP hive table meta info

In [11]:
table = "ucc_shipping_addr_chargeback_production_verification.ucc_shipping_addr_chargeback_history"
ret = spark.sql(f"DESCRIBE TABLE EXTENDED {table}")

for r in ret.where("col_name = 'Location'").select("data_type").take(1):
    print(r)


Row(data_type='gs://pypl-bkt-rsh-row-std-gds-art/user/root/bsi_verification_dbs/ucc_shipping_addr_chargeback_production_verification/ucc_shipping_addr_chargeback_history')